The project related to image cropping is not yet included in this file.

https://github.com/spumonie/eyeballgang

This file utilizes the thumbnail of the original images without any 
preprocessing.

In [1]:
import packages.small_batch_images as sbim
import pandas as pd
import numpy as np

df = sbim.load_annotation()
# Currently we consider only the healthy people
df_healthy = df[df['N']==1]
df_healthy.head()

ModuleNotFoundError: No module named 'pandas'

In [29]:
df_healthy.shape

(1040, 14)

In [50]:
dataset = pd.DataFrame(columns=['Age', 'LR', 'LG', 'LB', 'RR', 'RG', 'RB'])
for i in range(len(df_healthy)):
    sample = df_healthy.iloc[i]
    sample = sample[:1] # pick ID and age only

    # store the info of the left eye
    img = sbim.load_left_eye_image(class_='N', df = df_healthy, i=i)
    img = np.array(img)
    # store the averaged number for each color channel
    colors =  ['R', 'G', 'B']
    for j in range(len(colors)):
        sample['L'+ colors[j]] = img[:, :, j].mean()

    # same for the right eye
    img = sbim.load_right_eye_image(class_='N', df = df_healthy, i=i)
    img = np.array(img)
    colors =  ['R', 'G', 'B']
    for j in range(len(colors)):
        sample['R'+ colors[j]] = img[:, :, j].mean()

    # add the current sample to the dataset
    dataset.loc[len(dataset)] = sample

dataset.head()

,Age,LR,LG,LB,RR,RG,RB
0,57,71.801359,50.820318,34.355798,70.800345,49.648224,30.825615
1,59,115.076816,50.479862,29.224383,124.850529,54.599871,31.795094
2,51,123.61011,72.155129,49.583021,117.693821,68.319058,48.042853
3,51,53.220355,36.534905,21.677891,62.979265,41.060657,20.295689
4,63,75.193325,54.748073,32.905087,66.127949,45.4379,26.157673


In [52]:
# save the dataset for further tuning
dataset.to_csv('outputs/RGB_mean_healthy.csv')

In [56]:
dataset = dataset.to_numpy()

# easiest train and test split
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset)

# linear regression model
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()


In [61]:
lr_model.fit(train[:, 1:], train[:, 0])
# R2 score
lr_model.score(train[:, 1:], train[:, 0])

0.006597684645467838

In [73]:
# R2 score
lr_model.score(test[:, 1:], test[:, 0])

-0.011525631334656117

In [75]:
# print the test dataset accuracy
test_y_pred = np.round(lr_model.predict(test[:, 1:]))
print((np.sum(test_y_pred == test[:, 0])/len(test[:, 0]))*100)

2.307692307692308


2% of the test data are correctly labeled

In [77]:
from numpy.random import randint
print(randint(0, 100, size=100))

[61 36 24 59 87 64 36 37 93 91 75 58 72 86 90 93 62 15 69 69 27  1 88 91
 42 74  9 55 66 50 69 94 55 29 99 42 79 63 25 61 80  2 64 60 96 70 73  0
  7  6 74 61 91 62 55 35 77 60 19 67 22 67  5  4 26 24 44 84 35 94 17 59
 72 75 15 18 73 57 45 19 93  5 50 20 33 63 77 79 37 96 42 91 57 36 94 23
 38  9 81 35]


In [94]:
random_guess_model = randint(1, 89, size=len(test))
print(np.sum(random_guess_model == test[:, 0])/len(test[:, 0])*100)

1.9230769230769231


A random guess model also are not that promissing.